In [1]:
import nltk
import random
from openpyxl import  load_workbook
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

In [2]:
def get_most_common(data, n=200):
    # Finish stop words
    stop_words = set(stopwords.words('finnish'))
    
    # Join all words in our data
    full_text = ' '.join(w for w, _ in data)
    
    # Make all text lower
    full_text = full_text.lower()
    
    # Delete all chars that is not letter
    full_text = re.sub('[^a-z^ä^ö^é]', ' ', full_text)
    
    # Skip all words smaller than 2 symbols
    words = [word for word in full_text.split() if len(word) > 1]
    
    # Detele stop words 
    words = [word for word in words if word not in stop_words]
    
    # Steaming words
    finnish_stem = SnowballStemmer('finnish', ignore_stopwords=True)
    words = [finnish_stem.stem(word) for word in words]
    
    # Most common words
    return [w for w, _ in nltk.FreqDist(words).most_common(n)]

def document_features(document, word_features):
    document = document.lower()
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document)
    
    return features

def get_featureset(data, word_features):
    return tuple((document_features(d, word_features), c) for (d, c) in data)

In [3]:
wb = load_workbook("manual_all products_030917.xlsx")
sheet = wb['Sheet1']
cells = sheet['A:B']

random.seed = 0
random.shuffle(data)

NameError: name 'data' is not defined

In [13]:
class TextCleaner:
    
    def cleaner(self, text):
        
        stop_words = set(stopwords.words('finnish'))
        full_text = text.lower()
    
        # Delete all chars that is not letter
        #full_text = re.sub('[^a-z^ä^ö^é]', ' ', full_text)
        
        # Skip all words smaller than 2 symbols
        #words = [word for word in full_text.split() if len(word) > 1]

        # Detele stop words 
        #words = [word for word in words if word not in stop_words]

        # Steaming words
        #finnish_stem = SnowballStemmer('finnish', ignore_stopwords=True)
        
        #words = [finnish_stem.stem(word) for word in words]
        
        # todo: if empty words
        words = [full_text]
        return ' '.join(words)
    
    def transform(self, X):
        # Convert to ndarray 
        X_ndarray = np.array(X)

        # Vectorise function for cleaning text, only one column with text support
        func = np.vectorize(self.cleaner)
        
        # Return result
        return func(X_ndarray)

        
    def fit(self, X, y=None):
        return self

In [8]:
# [[...], [...]]
data = [[cell.value for cell in row] for row in cells]

# [[1,2], [1, 2] ...]
data = list(zip(*data))


data = np.array(data)
data = data[1:]
np.random.shuffle(data)

X, y = data[:, 0], data[:, 1]

In [14]:
# Model
stop_words = stopwords.words('finnish')
pipline = Pipeline([
    ('cln', TextCleaner()),
    ('vect', CountVectorizer(stop_words=stop_words, max_df=1e-1)),
    ('tfidf', TfidfTransformer(use_idf=True, smooth_idf=True)),
    ('clf', MultinomialNB(alpha=1e-6, fit_prior=True)),
])
pipline.fit(X, y)
print(pipline.score(X, y))

0.999240121581


In [82]:
def read_examples(path, sheet='Sheet1', product_column='A', category_column='B', offset=1):
    wb = load_workbook(path)
    sheet = wb[sheet]
    product_names = sheet[product_column]
    categories = sheet[category_column]
    cells = [product_names, categories]
    
    data = [[cell.value for cell in row] for row in cells]

    # [[1,2], [1, 2] ...]
    data = list(zip(*data))

    data = np.array(data)
    data = data[offset:]
    
    return data

def read_notclassified(path, sheet='Sheet1', product_column='A', offset=1):
    wb = load_workbook(path)
    sheet = wb[sheet]
    
    product_names = sheet[product_column]
    data = [c.value for c in product_names]
    
    data = data[offset:]
    return np.array(data)

def correct_decoding(data):
    func = np.vectorize(lambda s: s.replace('Ã¤', 'ä').replace('Ã¶', 'ö').replace('â‚¬', '').strip())
    return func(data)

In [56]:
notclassified = read_notclassified('products_to_classify_yevhenii.xlsx')
notclassified = correct_decoding(notclassified)

examples = read_examples('manual_all products_030917.xlsx')

In [83]:
s = 0

cleaned_prodcuts = cleaner_vect(examples[:, 0])
notfounded = []
result = []
for i, product in enumerate(notclassified):
    
    if i % 100 == 0: print(i, end='\t')
    
    matched_products = examples[examples[:, 0] == product]
    if len(matched_products) > 0:
        result.append([product, get_category(matched_products)])
        continue
    
    cleaned_current = cleaner(product)
    matched_products = examples[cleaned_prodcuts == cleaned_current]
    if len(matched_products) > 0:
        result.append([product, get_category(matched_products)])
        continue
        
    matched_products = examples[contains_words_vect(cleaned_prodcuts, cleaned_current)]
    if len(matched_products) > 0:
        result.append([product, get_category(matched_products)])
        continue
        
    
    result.append([product, predict_category(cleaned_prodcut)])
    
    
print()

0	100	200	300	400	500	600	700	800	900	1000	1100	1200	1300	1400	1500	1600	1700	1800	1900	2000	2100	2200	2300	2400	2500	2600	2700	2800	2900	3000	3100	3200	3300	3400	3500	3600	3700	3800	3900	4000	4100	4200	4300	4400	4500	4600	4700	4800	4900	5000	5100	5200	5300	5400	5500	5600	5700	5800	5900	6000	6100	6200	6300	6400	6500	6600	6700	6800	6900	7000	7100	7200	7300	7400	7500	7600	7700	7800	7900	8000	8100	8200	8300	8400	8500	8600	8700	8800	8900	9000	9100	9200	9300	9400	9500	9600	9700	9800	9900	10000	10100	10200	10300	10400	10500	10600	10700	10800	10900	11000	11100	11200	11300	11400	11500	11600	11700	11800	11900	12000	12100	12200	12300	12400	12500	12600	12700	12800	12900	13000	13100	13200	13300	13400	13500	13600	13700	13800	13900	14000	14100	14200	14300	14400	14500	14600	14700	14800	14900	15000	15100	15200	15300	15400	15500	15600	15700	15800	15900	16000	16100	16200	16300	16400	16500	16600	16700	16800	16900	17000	17100	17200	17300	17400	17500	17600	17700	17800	17900	18000	18100	18200	18300	18400	18

In [84]:
result

[['Romantiikkapaketti Lâ€™Ermitage hotellissa', 'Hemmottelu ja hyvinvointi'],
 ['***Viinikurssi - viini ja ruoka kahdelle', 'Ruokaelämykset'],
 ['Paskan palvelun illallinen', 'Ruokaelämykset'],
 ['*Kokovartalohieronta', 'Hemmottelu ja hyvinvointi'],
 ['Avoin rallikokeilupäivä', 'Ajoelämykset'],
 ['Lentosimulaattori - Kapteenikurssi', 'Lentoelämykset'],
 ['VIP Ajoelämys Lamborghinilla', 'Ajoelämykset'],
 ['25 elämystä isälle', 'Elämyspaketit'],
 ['SPA - jalkahoito', 'Hemmottelu ja hyvinvointi'],
 ['Lentosimulaattori - Lentäjäkurssi', 'Lentoelämykset'],
 ['Panimovierailu kahdelle', 'Ruokaelämykset'],
 ['Paskan palvelun illallinen', 'Ruokaelämykset'],
 ['Lahjakortti container | 300', 'Yleinen lahjakortti'],
 ['Illallinen pimeässä kahdelle Ravintola Sipuli', 'Ruokaelämykset'],
 ['Esittelykurssi kevytlentokoneella - Kevytilmailu.net', 'Lentoelämykset'],
 ['Illallinen pimeässä kahdelle Ravintola Sipuli', 'Ruokaelämykset'],
 ['Parisalsa aloittelijoille (1 kerta)', 'Kurssit'],
 ['HotStone kuum